<a href="https://colab.research.google.com/github/GeorgeU2030/analysis-rnn-lstm/blob/main/code-implementation/RNN_LSTM_TI2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sentiment analysis - RNN and LSTM.

Members

- Luis Botero
- Juan Medina
- George Trujillo

1. Gather the dataset for sentiment analysis from the UCI Machine Learning Repository, specifically
the Sentiment Labelled Sentences Data Set

In [5]:
! pip install scikeras

In [1]:
! pip install scikeras
import os
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, Sequential
from keras import layers
from keras.layers import LSTM, Activation, Embedding, Dense
from keras.utils import pad_sequences
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from collections import Counter
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luisb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luisb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. Preprocess the text data, including tokenization, lowercasing, and removing stopwords. Prepare the data for supervised learning (use NLTK).

In [3]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# We load the data from the websites
# amazon
df_amazon = pd.read_csv('sentiment labelled sentences/amazon_cells_labelled.txt', sep='\t', header=None)
df_amazon.columns = ['sentence', 'label']

# imdb
df_imdb = pd.read_csv('sentiment labelled sentences/imdb_labelled.txt', sep='\t', header=None)
df_imdb.columns = ['sentence', 'label']

# yelp
df_yelp = pd.read_csv('sentiment labelled sentences/yelp_labelled.txt', sep='\t', header=None)
df_yelp.columns = ['sentence', 'label']

# We mix the dataframes, reload the index, for no obtain duplicate indexes
df = pd.concat([df_amazon, df_imdb, df_yelp], ignore_index=True)

# We tokenize each sentence into individual words, convert words to lowercase, and remove stopwords
stop_words = set(stopwords.words('english'))

df['sentence'] = df['sentence'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word.isalnum() and word.lower() not in stop_words]))

print(df['sentence'])


0                                   way plug us unless go
1                                          good excellent
2                                                   great
3                   tied charger conversations lasting 45
4                                                     mic
                              ...                        
2743                            think food flavor texture
2744                                   appetite instantly
2745                           overall impressed would go
2746           whole experience think go ninja sushi next
2747    wasted enough life poured salt wound drawing t...
Name: sentence, Length: 2748, dtype: object


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# We prepare the data for supervised learning
# We divide the data in training and test

X_train, X_test, y_train, y_test = train_test_split(df['sentence'], df['label'], test_size=0.3, random_state=123)

# Convert the text to numerical characteristics
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Convert the categorical to numerical characteristics

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


### Otra forma de preprocesar los datos:

In [2]:
stop_words = set(stopwords.words('english'))
punctuation_chars = string.punctuation

# We load the data from the websites
# amazon
sentences_amazon = list(open('amazon_cells_labelled.txt', encoding='utf-8').readlines())


# imdb
sentences_imdb = list(open('imdb_labelled.txt', encoding='utf-8').readlines())


# yelp
sentences_yelp = list(open('yelp_labelled.txt', encoding='utf-8').readlines())

#junta las listas en una
inter=[*sentences_amazon, *sentences_imdb, *sentences_yelp]


#FUNCTION count unique words of a list of lists of words
def counter_words(text):
    count=Counter()
    for line in text:
        for word in word_tokenize(line):
            count[word] += 1
    return count



cleanedText=[]
results=[]
for i in inter:
    #separacion frase - valor
    sen_val=i.split("\t")
    #lowercase
    sen_val[0]=sen_val[0].lower()
    #quitar puntuacion
    cleanedLine=''.join([i for i in sen_val[0] if i not in punctuation_chars])
    #tokenizar, quitar stop words, re juntar la frase.
    words=word_tokenize(cleanedLine)
    cleanedWords=" ".join([i for i in words if i not in stop_words])
    #guardar frases limpias y valores
    results.append(int(sen_val[1]))
    cleanedText.append(cleanedWords)

print(cleanedText, results)

df=pd.DataFrame({'sentences': cleanedText, 'value': results})

print(df.sentences)

#creando dataframes de entreno y test
train_size=int(df.shape[0]*0.8)
train_df=df[:train_size]
test_df=df[train_size:]
print(train_df.shape[0], test_df.shape[0])

#cantidad de palabras distintas posibles
uniqueWords=len(counter_words(df.sentences))
print(uniqueWords)

#asociar cada palabra unica a un unmero
tokenizer=Tokenizer(num_words=uniqueWords)
tokenizer.fit_on_texts(train_df.sentences)
word_index = tokenizer.word_index
print(word_index)

#armar las secuencias de numeros quer representan las frases
train_sequences = tokenizer.texts_to_sequences(train_df.sentences)
test_sequences = tokenizer.texts_to_sequences(test_df.sentences)

#print(train_sequences)


#estandarizando el largo de las frases a 20 palabras
maxLenWords=20
train_padded = pad_sequences(train_sequences, maxlen=maxLenWords, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=maxLenWords, padding='post', truncating='post')
print(train_padded)


['way plug us unless go converter', 'good case excellent value', 'great jawbone', 'tied charger conversations lasting 45 minutesmajor problems', 'mic great', 'jiggle plug get line right get decent volume', 'several dozen several hundred contacts imagine fun sending one one', 'razr owneryou must', 'needless say wasted money', 'waste money time', 'sound quality great', 'impressed going original battery extended battery', 'two seperated mere 5 ft started notice excessive static garbled sound headset', 'good quality though', 'design odd ear clip comfortable', 'highly recommend one blue tooth phone', 'advise everyone fooled', 'far good', 'works great', 'clicks place way makes wonder long mechanism would last', 'went motorolas website followed directions could get pair', 'bought use kindle fire absolutely loved', 'commercials misleading', 'yet run new battery two bars thats three days without charging', 'bought mother problem battery', 'great pocket pc phone combination', 'ive owned phone 7 

# DUMMY CLASSIFIER

3. Implement a DummyClassifier. Train the model on the preprocessed data and evaluate its
performance in terms of accuracy, precision, recall, and F1-score. Use these results as the baseline

In [5]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# We create the Dummy Classifier
dummy_classifier = DummyClassifier(strategy="most_frequent")

# We train the model with the data preprocessed
dummy_classifier.fit(X_train_vectorized, y_train_encoded)

# We do the predictions with the test data
y_pred = dummy_classifier.predict(X_test_vectorized)

# We evaluate the performance - accuracy - precision - recall - f1 score
accuracy = accuracy_score(y_test_encoded, y_pred)
precision = precision_score(y_test_encoded, y_pred)
recall = recall_score(y_test_encoded, y_pred)
f1 = f1_score(y_test_encoded, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.4921212121212121
Precision: 0.4921212121212121
Recall: 1.0
F1-score: 0.6596263200649878


# RNN

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical

#Once the stopwords are removed, tokenize and set to lowercase
tokenizer = Tokenizer(num_words=1000, lower=True)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

#Generate arrays of length:100 of the reviews so they're all of the same length.
maxlen=100
X_train = pad_sequences(X_train, padding='post',maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post',maxlen=maxlen)

#Set dependent variable to Categorical labels
num_classes=2
y_train = to_categorical(y_train,num_classes=num_classes)
y_test = to_categorical(y_test,num_classes=num_classes)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, SimpleRNN, Embedding
from keras import optimizers
import numpy as np

#Transformation to a tri-dimensional vector
X_train = np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1)
X_test = np.array(X_test).reshape(X_test.shape[0],X_test.shape[1],1)

#Reshape of dependent variable so it can be received by the model
y_train = y_train.reshape(-1, 1)
y_train = np.array(y_train)
# Verify and adjustment of the size of y_train so it matches X_train
y_train = y_train[:X_train.shape[0]]


# Embedding Size, vectorial size of the vocabulary words
embedding_dim = 32

# RNN Model with 8 neurons
# Dense layer ends up giving 1 result and uses sigmoid for this bi-categorical problem
model = Sequential([
    Embedding(tokenizer.num_words, embedding_dim, input_length=maxlen),
    SimpleRNN(8),
    Dense(1, activation='sigmoid')
])

In [ ]:
#Compilation and fitting of the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(
    X_train, y_train, epochs=30
)

In [ ]:
#Test on test values to check accuracy
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_test)
y_test_ = np.argmax(y_test, axis=1)

# Rounds predictions from 0 to 1 as it is a binary Classification problem
y_pred_binary = np.round(y_pred)

# Calculates accuracy
accuracy = accuracy_score(y_test_, y_pred_binary)
print(accuracy)


In [ ]:
#Use of gridSearch to improve model
from keras.layers import Embedding, SimpleRNN, Dense
from keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from sklearn.model_selection import train_test_split

# Define function create_model to check hyperparameters

def create_model(embedding_dim, rnn_units, optm):
    model = Sequential([
        Embedding(input_dim=1000, output_dim=embedding_dim, input_length=maxlen),
        SimpleRNN(units=rnn_units),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optm, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def evaluate_model(model, X, y):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, np.round(y_pred))
    precision = precision_score(y, np.round(y_pred))
    recall = recall_score(y, np.round(y_pred))
    f1 = f1_score(y, np.round(y_pred))
    kappa = cohen_kappa_score(y, np.round(y_pred))
    return accuracy, precision, recall, f1, kappa

# Create the KerasClassifier model with some parameters
param_grid = {'embedding_dim': [8, 16, 32], 'rnn_units': [4, 8, 16], 'optm': ['adam', 'rmsprop']}

keras_model = KerasClassifier(build_fn=create_model, batch_size=32,verbose=0, epochs=3,embedding_dim=16, rnn_units=8, optm='adam')

# Configure GridSearchCV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, scoring='accuracy', cv=cv)
grid_result = grid.fit(X_train, y_train)

#Results of best configuration
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: {:.2f}".format(grid_result.best_score_))

In [ ]:
# Test of the model
a= ['I hate gatorade.'] 
a = tokenizer.texts_to_sequences (a)
a=np.array(a)
a = pad_sequences (a, padding='post', maxlen=maxlen)
a = a.reshape((a.shape[0], a.shape[1], 1))
print(a.shape)
prediction = model.predict(np.array(a))
print(prediction) # 0 means negative and 1 means positive and by how far or close it is to one or the other we can evaluate the model

# LSTM  

Implementación del lstm a través de los datos procesados por 

In [11]:

#modelo RNN LSTM
model = keras.models.Sequential()
model.add(Embedding(uniqueWords, 36, input_length=maxLenWords))
model.add(layers.Bidirectional(LSTM(16)))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

#tesumen de la estructura del modelo
model.summary()

#earlyStopping para que no sigam mas epochs si no es necesario 
checkpoint_filepath = os.getcwd()
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_loss', mode='min', save_best_only=True)
callbacks = [EarlyStopping(patience=2), model_checkpoint_callback]


#compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

#entrenar el modelo
history =  model.fit(train_padded, train_df.value, epochs=3, validation_data=(test_padded, test_df.value), callbacks=callbacks)

#devuelve lista de listas de 1 elemento decimal, por eso se usa el elemento 0 decada sublista, y se redondea.
y_pred=[round(i[0], 0) for i in model.predict(test_padded)]


# accuracy - precision - recall - f1 score
accuracy = accuracy_score(test_df.value, y_pred)
precision = precision_score(test_df.value, y_pred)
recall = recall_score(test_df.value, y_pred)
f1 = f1_score(test_df.value, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 36)            189972    
                                                                 
 bidirectional_7 (Bidirecti  (None, 32)                6784      
 onal)                                                           
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dense_15 (Dense)            (None, 1)                 17        
                                                                 
Total params: 197301 (770.71 KB)
Trainable params: 197301 (770.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/3
71/75 [===========================>..] - ETA: 0s - loss: 0.6859 - accu

INFO:tensorflow:Assets written to: c:\Users\luisb\OneDrive\Escritorio\ICESI\S6\Discretas3\TI2\analysis-rnn-lstm\code-implementation\assets


75/75 [==============================] - 11s 111ms/step - loss: 0.6844 - accuracy: 0.6017 - val_loss: 0.6618 - val_accuracy: 0.6767
Epoch 2/3
75/75 [==============================] - ETA: 0s - loss: 0.4740 - accuracy: 0.8271INFO:tensorflow:Assets written to: c:\Users\luisb\OneDrive\Escritorio\ICESI\S6\Discretas3\TI2\analysis-rnn-lstm\code-implementation\assets


INFO:tensorflow:Assets written to: c:\Users\luisb\OneDrive\Escritorio\ICESI\S6\Discretas3\TI2\analysis-rnn-lstm\code-implementation\assets


75/75 [==============================] - 7s 94ms/step - loss: 0.4740 - accuracy: 0.8271 - val_loss: 0.4705 - val_accuracy: 0.7917
Epoch 3/3
19/19 [==============================] - 1s 2ms/step
0.3:  0.0
0.7:  1.0
0.5:  0.0
Accuracy: 0.7766666666666666
Precision: 0.7255520504731862
Recall: 0.8303249097472925
F1-score: 0.7744107744107744


In [3]:


# funcion para crear modelo a partir de parametros
def create_lstm_model(units_lstm, units_dense, embedding_dim):
    model = Sequential()
    model.add(Embedding(input_dim=uniqueWords, output_dim=embedding_dim, input_length=maxLenWords))
    model.add(LSTM(units=units_lstm))
    model.add(Dense(units_dense, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create KerasClassifier para poder aplicar gridSearchCV
model = KerasClassifier(build_fn=create_lstm_model, units_lstm=8, units_dense=4, embedding_dim=12, epochs=8, batch_size=32, verbose=0)

# Hyperparametros
param_grid = {
    'units_lstm': [16, 32, 64],
    'units_dense': [8, 16, 32],
    'embedding_dim': [12, 24, 36],
}

# clasidicacion de tareas
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='accuracy')

# GridSearch
grid_result = grid.fit(train_padded, train_df.value)

print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: {:.2f}".format(grid_result.best_score_))

C:\Users\luisb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


C:\Users\luisb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\luisb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\luisb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initia

Best Parameters:  {'embedding_dim': 36, 'units_dense': 16, 'units_lstm': 16}
Best Accuracy: 0.77
